In [607]:
import numpy as np
import pandas as pd

In [608]:
class Competency:
    def __init__(self, name, scoring_dict, min_val, max_val):
        """
        Initialize a Competency object.

        Args:
            name (str): The name of the criteria.
            min_val (float): The minimum value for the criteria.
            max_val (float): The maximum value for the criteria.
        """
        self.name = name
        self.scoring_dict = scoring_dict
        self.min_val = min_val
        self.max_val = max_val
        
    def numerical_normalize(self, x):
        '''
        Normalize a numerical form of a data to have values between 0 and 1.

        Args:
            x (float): The input data to normalize.

        Returns:
            normalized data (float): The normalized data.
        '''

        return (x - self.min_val) / (self.max_val - self.min_val)
    
    
    def nominal_normalize(self, entered_comp):
        x = self.scoring_dict[entered_comp]
       
        return (x - self.min_val) / (self.max_val - self.min_val)

In [609]:
class Item:
    def __init__(self, name, min_val, max_val ,item_type = "CONTINUOUS"):
        """
        Initialize a Item object.

        Args:
            name (str): The name of the Item.
            min_val (float): The minimum value for the Item.
            max_val (float): The maximum value for the Item.
            var_type: BINARRY, INTEGER ,and CONTINUOUS
        """
        self.name = name
        self.min_val = min_val
        self.max_val = max_val
        self.item_type = item_type
        
    def numerical_normalize(self, x):
        '''
        Normalize a numerical form of a data to have values between 0 and 1.

        Args:
            x (float): The input data to normalize.

        Returns:
            normalized data (float): The normalized data.
        '''

        return (x - self.min_val) / (self.max_val - self.min_val)    

In [610]:
edu_dict= {"None":1,"Diploma":2,"BSC":4,"MSC":6,"PHD":8}
lan_dict= {"low":1,"medium":2,"good":3,"advanced":4,"native":5}
gen_dict={"low":1,"medium":2,"good":4,"high":8,"advanced":10}
exp_dict={"0-5":1,"5-10":2,"10-15":4,"15-20":6,"20 and more":8}
psy_dict = {"low":1,"medium":2,"good":4,"high":8,"excellent":10}

In [611]:
Education = Competency("edaucation", edu_dict, min_val = 1, max_val = 8)
Language = Competency("language", lan_dict, min_val = 1, max_val = 5)
General = Competency("general", gen_dict, min_val = 1, max_val = 10)
Experience = Competency("experience", exp_dict, min_val = 1, max_val = 8)
Psycology = Competency("psycology_test", psy_dict, min_val = 1, max_val = 10)

In [612]:
Salary = Item("salary", min_val = 5, max_val = 25, item_type = "CONTINUOUS")
Bonus = Item("bonus", min_val = 0, max_val = 5, item_type = "CONTINUOUS")
Flex = Item("flexibale hours", min_val = 0, max_val = 4, item_type = "INTEGER")
Remote = Item("remotabilty", min_val =0 , max_val = 1, item_type = "BINARY")

In [613]:
class Candidate :    
    def __init__(self ,name ,education, english, general_skill, experience, psycho_test, items_weight_list):
        self.name =name
        
        self.items_weight_list = items_weight_list
        self.initial_organs =[]
        self.available_organs = {}
        self.selected_organ_so_far = None
        
        # Matching Property
        
        self.current_timestep = 0
        
        # Competencies
        # Take a note that this values are normailized
        self.edu = Education.nominal_normalize(education)
        self.lan = Language.nominal_normalize(english)
        self.gen = General.nominal_normalize(general_skill)
        self.exp = Experience.nominal_normalize(experience)
        self.psy = Psycology.nominal_normalize(psycho_test)
             
    def preprocess_organs(self):
        for org in self.initial_organs:
            
            self.available_organs[org.name] = {}
                    
        
    def calculate_organ_utility(self, Organ, explicit_utility):
        
        if Organ.IS_UNDER_INVESTIGATION == False:
            
            weight_list = self.items_weight_list        
            Up = weight_list[0]* Organ.salary+ weight_list[1]*Organ.bonus + weight_list[2]* Organ.flexibality+ weight_list[3]*Organ.remotability
        
        if Organ.IS_UNDER_INVESTIGATION == True:
            Up = explicit_utility
        
        self.available_organs[Organ.name]["utility'"] = Up
        self.available_organs[Organ.name]["interviewd"] = False
        
        
    def choose_best_available_organ(self):
        
        if len(self.initial_organs) > 0:
            
            
            max_founded_utility = 0
            
            
            for org in self.initial_organs:
                
                
                if self.available_organs[org.name]["utility'"] >= max_founded_utility and org.available_candidates[self.name]["Rejected"] == False:
                    
                    max_founded_utility = self.available_organs[org.name]["utility'"]
                    
                    
                    
            
            
            for org in self.initial_organs:
                if self.available_organs[org.name]["utility'"] == max_founded_utility and org.available_candidates[self.name]["Rejected"] == False: 
                    
                
                    self.selected_organ_so_far = org
                    
                    
                    
                    
            if self not in self.selected_organ_so_far.recieved_resumes :
                # this condition avoids duplicate appendings
                self.selected_organ_so_far.recieved_resumes.append(self)
                self.available_organs[org.name]["interviewd"] = True
            
            return self.selected_organ_so_far
            
        else:
            return None
        
        

In [614]:
class Organization :
    def __init__(self, name,competencies_weight_list, salary = 5, bonus= 0, flexibality= 0, remotability= 0,  IS_UNDER_INVESTIGATION = False):
        self.name = name
        
        self.IS_UNDER_INVESTIGATION = IS_UNDER_INVESTIGATION
        self.competencies_weight_list = competencies_weight_list
        self.initial_candidates = []
        self.available_candidates = {}
        self.recieved_resumes = []
        self.selected_candidate_so_far = None
        self.initial_utility_gained = 0
        # Matching Property
        self.current_timestep = 0
        
        # Organiztion's Job offer package items:
        self.salary = Salary.numerical_normalize(salary)
        self.bonus = Bonus.numerical_normalize(bonus)
        self.flexibality = Flex.numerical_normalize(flexibality)
        self.remotability = Remote.numerical_normalize(remotability)
        
    def preprocess_candidates(self):
        for can in self.initial_candidates:
            self.available_candidates[can.name] = {}
            
            
    def calculate_candidate_utility(self, Candidate):
        weight_list = self.competencies_weight_list        
        U = weight_list[0]* Candidate.edu+ weight_list[1]*Candidate.lan + weight_list[2]*Candidate.gen + weight_list[3]*Candidate.exp + weight_list[4]*Candidate.psy
        
        self.available_candidates[Candidate.name]["utility"] = U
        self.available_candidates[Candidate.name]["interviewed"] = False
        self.available_candidates[Candidate.name]["Rejected"] = False
        
    def choose_the_best_candidate_intending(self):
    
        if len(self.recieved_resumes) > 0:
   #         
            max_founded_utility = self.initial_utility_gained
            for can in self.recieved_resumes:
    #            
                # The Organ interviews with every intending candidates
                self.available_candidates[can.name]["interviewed"] = True
     #           
                if self.available_candidates[can.name]["utility"] >= max_founded_utility:
                    # This condition controls whether the candidate is selected or rejected, 
                    # If selected, the organ keep the selected candidate in its waiting list.
                    # if rejected, the organ is no longer accessilbe for the rejected candidate.
      #              
                    max_founded_utility = self.available_candidates[can.name]["utility"]
        
        
  
        
        
            for can in self.recieved_resumes:
                if self.available_candidates[can.name]["utility"] == max_founded_utility: 
                    
                  
                    self.selected_candidate_so_far = can
                    self.initial_utility_gained = max_founded_utility 
                    
       #         
                else:
                   
                    self.available_candidates[can.name]["Rejected"] = True
                    
            
                '''
                if self in can.initial_organs: #and not (self.selected_candidate_so_far == can):
                        print("Organ {:} B==================D Candidate {:}".format(self.name,can.name ))
                        
                        self.available_candidates[can.name]["status"] = "Rejected"
                        can.initial_organs.remove(self)
                '''
                    
                    # Maybe another organ deserves him/her more :)
        #            
            # This one was forgotten. The Organ only keeps selected candidates in its waiting list   
            self.recieved_resumes = [self.selected_candidate_so_far]
            
            if self != self.selected_candidate_so_far.selected_organ_so_far :
                # this condition avoids duplicate assignment
                self.selected_candidate_so_far.selected_organ_so_far = self 
         #   
            return self.selected_candidate_so_far
            
        else:
            return None
        

In [615]:
class Matcher:
    def __init__(self, name, intending_side, intended_side, explicit_utility):
        
        self.name = name
        self.intending_side = intending_side
        self.intended_side = intended_side
        self.explicit_utility = explicit_utility
        
        self.events = []
        self.matched_list = []      
        
    def public_introduction(self):
        # In this function, all of the sides are introduced to each other
        
        for org in self.intended_side:
            org.initial_candidates = self.intending_side 

        for can in self.intending_side:
            can.initial_organs = self.intended_side 
    
        for org in self.intended_side:
            org.preprocess_candidates()
            for can in org.initial_candidates:
                org.calculate_candidate_utility(can)

        for can in self.intending_side:
            can.preprocess_organs()
            for org in can.initial_organs:

                can.calculate_organ_utility(org, self.explicit_utility)
                
    
    
    def is_matching_over(self):
        # This function checks whether the matching is over
        # the matching is over if the two last matches of time steps are the same,
        
        if len(self.matched_list) >= 2:
        
            current_matches = set(self.matched_list[-1])
            pervious_matches = set(self.matched_list[-2])
            
            if current_matches == pervious_matches:
                return True
            else:
                return False
        else:
            return False
    
    def start_matching(self):
        
        time_step = 0
        while (self.is_matching_over() == False) :
            
            print(time_step)
            time_step += 1
            
            temp_matches = []
            
            
           
            for can in self.intending_side:
                
                
                selected_org = can.choose_best_available_organ()
                
                if selected_org != None:
                    self.events.append("Candidate {:} sent his/her resume to Organ {:} on time step {:}".format(can.name,selected_org.name, time_step))
                else :
                    self.events.append("Candidate {:} can't send any resumes on time step {:}".format(can.name, time_step))
                
            for org in self.intended_side:
                selected_can = org.choose_the_best_candidate_intending()
                
                if selected_can != None:
                    self.events.append("Organ {:} kept Candidate {:} and rejected the others on time step {:}".format(org.name,selected_can.name, time_step))
                    temp_matches.append((org.name, selected_can.name))
                else:
                    self.events.append("Organ {:} has recieved no resume on time step {:}".format(org.name, time_step))
                    #temp_matches.append((org.name, selected_can))
                 
            self.matched_list.append(temp_matches)     
                

In [616]:
C1 = Candidate("Shayan" , "MSC", "native" , "good" , "0-5" , "good",[0.5,0.2,0.2,0.1])
C2 = Candidate("Mamad" , "PHD", "medium" , "medium" , "10-15" , "low",[0.4,0.2,0.15,0.25])
C3 = Candidate("Sadra" , "BSC", "good" , "high" , "0-5" , "medium",[0.35,0.1,0.25,0.3])
C4 = Candidate("Ehsan" , "PHD", "advanced" , "advanced" , "15-20" , "excellent",[0.6,0,0.3,0.1])
C5 = Candidate("Gorz-Ali" , "BSC", "advanced" , "high" , "0-5" , "good",[0.5,0.2,0.15,0.15])

In [617]:
#name,competencies_weight_list, salary = 5, bonus= 0, flexibality= 0, remotability= 0,  IS_UNDER_INVESTIGATION = False
O1 = Organization("DigiKala",competencies_weight_list=[0.4,0.1,0.1,0.25,0.15], salary = 5, bonus= 0, flexibality= 0, remotability= 0, IS_UNDER_INVESTIGATION = True)
O2 = Organization("Snapp",competencies_weight_list=[0.36,0.04,0.2,0.3,0.1], salary = 9, bonus= 4, flexibality= 2, remotability= 0, )
O3 = Organization("National-Petrolium",competencies_weight_list=[0.31,0.14,0.16,0.14,0.25], salary = 8, bonus= 5, flexibality= 1, remotability= 1, )

In [618]:
initial_organs = [O1, O2, O3]
initial_candidates = [C1 ,C2 ,C3 ,C4 ,C5] 

In [619]:
match = Matcher("Match_1", intending_side = initial_candidates, intended_side = initial_organs, explicit_utility = 0.9)

In [620]:
match.public_introduction()

In [621]:
match.start_matching()

0
1
2
3


In [622]:
match.events

['Candidate Shayan sent his/her resume to Organ DigiKala on time step 1',
 'Candidate Mamad sent his/her resume to Organ DigiKala on time step 1',
 'Candidate Sadra sent his/her resume to Organ DigiKala on time step 1',
 'Candidate Ehsan sent his/her resume to Organ DigiKala on time step 1',
 'Candidate Gorz-Ali sent his/her resume to Organ DigiKala on time step 1',
 'Organ DigiKala kept Candidate Ehsan and rejected the others on time step 1',
 'Organ Snapp has recieved no resume on time step 1',
 'Organ National-Petrolium has recieved no resume on time step 1',
 'Candidate Shayan sent his/her resume to Organ National-Petrolium on time step 2',
 'Candidate Mamad sent his/her resume to Organ National-Petrolium on time step 2',
 'Candidate Sadra sent his/her resume to Organ National-Petrolium on time step 2',
 'Candidate Ehsan sent his/her resume to Organ DigiKala on time step 2',
 'Candidate Gorz-Ali sent his/her resume to Organ National-Petrolium on time step 2',
 'Organ DigiKala kept 

In [623]:
print("The Stable Pairs of Matching are: ",match.matched_list[-1])

The Stable Pairs of Matching are:  [('DigiKala', 'Ehsan'), ('Snapp', 'Mamad'), ('National-Petrolium', 'Shayan')]
